In [1]:
import os
import pandas as pd
import pickle
import biotype.similarity as sim

In [2]:
DATA_DIR_PATH = "datasets/export"
META_FILE_PATH = "datasets/meta.csv"
PEAK_DIR_PATH = "extracted_peaks"
files =  os.listdir(DATA_DIR_PATH)
files = [fn  for fn in files if fn.endswith(".txt")]
files = sorted(files)
for file in files[:5] :
    print(file)

L100_0_G7_1.txt
L100_0_G8_1.txt
L101_0_A1_1.txt
L101_0_A2_1.txt
L102_0_A3_1.txt


In [5]:
import random
a = [(v+5 .0* random.random(), random.choice(range(3))) for _ in range(3) for v in range(1230,1240)]
for x in sorted(a):
    print(x)

(1230.3305161679914, 1)
(1231.0130551994055, 0)
(1231.3532787868435, 0)
(1232.4255209716428, 0)
(1232.5163361722357, 1)
(1233.3396771133725, 2)
(1233.8016473323844, 2)
(1233.9990113138301, 1)
(1234.3390100903496, 1)
(1235.4391228827146, 2)
(1235.8639516929545, 0)
(1235.9431871561662, 2)
(1236.0229919852222, 0)
(1236.0745568566392, 0)
(1236.1715408854086, 1)
(1236.2199572176012, 2)
(1236.957384206811, 1)
(1237.2143160486874, 0)
(1238.1513996367853, 2)
(1238.2106597093036, 0)
(1238.3624897289405, 1)
(1238.891537804621, 2)
(1239.6425706971131, 0)
(1239.6626512534358, 2)
(1239.7664107975106, 0)
(1240.0205220051914, 2)
(1240.780916547018, 0)
(1242.0569796436564, 2)
(1242.5024199328914, 2)
(1242.6138859451648, 0)


In [ ]:
n = len(files)
for i in range(n):
    df = None
    df = pd.read_table(f"{DATA_DIR_PATH}/{files[i]}",sep=" ", header=None,names=['m/z', 'intensity']) 
    x, y = df['m/z'].to_numpy(), df['intensity'].to_numpy()

    pickle_file = files[i][:-4] # trim '.txt'
    with open(f'{PEAK_DIR_PATH}/{pickle_file}_peaks.pkl', 'rb') as peak_file:
        peaks = pickle.load(peak_file)
        stype = files[i].split('_')[0]
        st = serotype.get(stype, 'NA')
        print(f'{pickle_file} ({st}) peaks: {len(peaks)}')
        peaks_extracted +=  [(x[peaks], y[peaks])] 